<a href="https://colab.research.google.com/github/convin305/sum-tudy/blob/master/daycon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 수정사항
#### 이름 : 수정사항 을 수정 후 기록하기
##### 만약에 cvs제출하면 맨 밑에 파일이름 뒤에 숫자 하나씩 증가시키겠습니다~!  
##### (팀장이 제출하라해서...제가 제출할때마다 바꿔서 할게요)
_____________________________  
* 수민 : resnet18을 resnet50으로 변경, cuda 메모리 부족으로 아직 돌리지는 않음

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from google.colab import output

In [ ]:
!mkdir "./dirty_mnist"

mkdir: cannot create directory ‘./dirty_mnist’: File exists


In [ ]:
!unzip "/content/drive/MyDrive/Data/dirty_mnist_2nd.zip" -d "./dirty_mnist/"

Archive:  /content/drive/MyDrive/Data/dirty_mnist_2nd.zip
replace ./dirty_mnist/00000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!mkdir "./test_dirty_mnist"
!unzip "/content/drive/MyDrive/Data/test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"

mkdir: cannot create directory ‘./test_dirty_mnist’: File exists
Archive:  /content/drive/MyDrive/Data/test_dirty_mnist_2nd.zip
replace ./test_dirty_mnist/50000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
dirty_mnist_answer = pd.read_csv("/content/drive/MyDrive/Data/dirty_mnist_2nd_answer.csv")
namelist = os.listdir('./dirty_mnist/')

In [ ]:
class ToTensor(object):
  def __call__(self, sample):
    image, label = sample['image'], sample['label']
    image = image.transpose((2, 0, 1))
    return {'image': torch.FloatTensor(image),
            'label': torch.FloatTensor(label)}

In [ ]:
to_tensor = T.Compose([
                       ToTensor()
])

In [ ]:
class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms=to_tensor,#미리 선언한 to_tensor를 transforms로 받음
                 augmentations=None):
        
        self.dir_path = dir_path # 데이터의 이미지가 저장된 디렉터리 경로
        self.meta_df = meta_df # 데이터의 인덱스와 정답지가 들어있는 DataFrame

        self.transforms = transforms# Transform
        self.augmentations = augmentations # Augmentation
        
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        # 폴더 경로 + 이미지 이름 + .png => 파일의 경로
        # 참고) "12".zfill(5) => 000012
        #       "146".zfill(5) => 000145
        # cv2.IMREAD_GRAYSCALE : png파일을 채널이 1개인 GRAYSCALE로 읽음
        image = cv2.imread(self.dir_path +\
                           str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
                           cv2.IMREAD_GRAYSCALE)
        # 0 ~ 255의 값을 갖고 크기가 (256,256)인 numpy array를
        # 0 ~ 1 사이의 실수를 갖고 크기가 (256,256,1)인 numpy array로 변환
        image = (image/255).astype('float')[..., np.newaxis]

        # 정답 numpy array생성(존재하면 1 없으면 0)
        label = self.meta_df.iloc[index, 1:].values.astype('float')
        sample = {'image': image, 'label': label}

        # transform 적용
        # numpy to tensor
        if self.transforms:
            sample = self.transforms(sample)

        # sample 반환
        return sample

모델 구성

In [ ]:
class MultiLabelResnet(nn.Module):
    def __init__(self):
        super(MultiLabelResnet, self).__init__()
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        self.resnet = models.resnet50() 
        self.FC = nn.Linear(1000, 26)

    def forward(self, x):
      x = F.relu(self.conv2d(x))
      x = F.relu(self.resnet(x))
      x = torch.sigmoid(self.FC(x))
      return x

In [ ]:
model = MultiLabelResnet()
model

MultiLabelResnet(
  (conv2d): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (re

학습

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
best_models = []
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(dirty_mnist_answer),1):
  print(f'[fold: {fold_index}]')
  torch.cuda.empty_cache()
  train_answer = dirty_mnist_answer.iloc[trn_idx]
  test_answer  = dirty_mnist_answer.iloc[val_idx]
  #Data set
  train_dataset = DatasetMNIST("dirty_mnist/", train_answer)
  valid_dataset = DatasetMNIST("dirty_mnist/", test_answer)
  #Data Loader
  train_data_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 128,shuffle = False,num_workers = 3)
  valid_data_loader = torch.utils.data.DataLoader(valid_dataset,batch_size = 32,shuffle = False,num_workers = 3)

  model = MultiLabelResnet()
  model.to(device)

  optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
  lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size = 5,gamma = 0.75)
  criterion = torch.nn.BCELoss()

  valid_acc_max = 0
  for epoch in range(10):
    train_acc_list = []
    with tqdm(train_data_loader,total=train_data_loader.__len__()) as train_bar:
      for sample in train_bar:
        train_bar.set_description(f"Train Epoch {epoch}")
        optimizer.zero_grad()
        images, labels = sample['image'], sample['label']
        images = images.to(device)
        labels = labels.to(device)

        model.train()
        with torch.set_grad_enabled(True):
          probs  = model(images)
          loss = criterion(probs, labels)
          loss.backward()
          optimizer.step()

          probs  = probs.cpu().detach().numpy()
          labels = labels.cpu().detach().numpy()
          preds = probs > 0.5
          batch_acc = (labels == preds).mean()
          train_acc_list.append(batch_acc)
          train_acc = np.mean(train_acc_list)

        train_bar.set_postfix(train_loss= loss.item(),train_acc = train_acc)
    valid_acc_list = []
    with tqdm(valid_data_loader,total=valid_data_loader.__len__(),unit="batch") as valid_bar:
      for sample in valid_bar:
        valid_bar.set_description(f"Valid Epoch {epoch}")
        optimizer.zero_grad()
        images, labels = sample['image'], sample['label']
        images = images.to(device)
        labels = labels.to(device)

        model.eval()

        with torch.no_grad():
          probs  = model(images)
          valid_loss = criterion(probs, labels)

          probs  = probs.cpu().detach().numpy()
          labels = labels.cpu().detach().numpy()
          preds = probs > 0.5
          batch_acc = (labels == preds).mean()
          valid_acc_list.append(batch_acc)

        valid_acc = np.mean(valid_acc_list)
        valid_bar.set_postfix(valid_loss = valid_loss.item(),valid_acc = valid_acc)

    lr_scheduler.step()
    if valid_acc_max < valid_acc:
       valid_acc_max = valid_acc
       best_model = model
       MODEL = "resnet50"
       path = "/content/drive/MyDrive/Daycon"
       torch.save(best_model, f'{path}{fold_index}_{MODEL}_{valid_loss.item():2.4f}_epoch_{epoch}.pth')

best_models.append(best_model)

NameError: ignored

In [ ]:
sample_images = images.cpu().detach().numpy()
sample_prob = probs
sample_labels = labels

idx = 1
plt.imshow(sample_images[idx][0])
plt.title("sample input image")
plt.show()

print('예측값 : ',dirty_mnist_answer.columns[1:][sample_prob[idx] > 0.5])
print('정답값 : ', dirty_mnist_answer.columns[1:][sample_labels[idx] > 0.5])

In [ ]:
#test Dataset 정의
sample_submission = pd.read_csv("/content/drive/MyDrive/Data/sample_submission.csv")
test_dataset = DatasetMNIST("test_dirty_mnist/", sample_submission)
batch_size = 128
test_data_loader = DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)

In [ ]:
predictions_list = []
# 배치 단위로 추론
prediction_df = pd.read_csv("/content/drive/MyDrive/Data/sample_submission.csv")

# 5개의 fold마다 가장 좋은 모델을 이용하여 예측
for model in best_models:
    # 0으로 채워진 array 생성
    prediction_array = np.zeros([prediction_df.shape[0],
                                 prediction_df.shape[1] -1])
    for idx, sample in enumerate(test_data_loader):
        with torch.no_grad():
            # 추론
            model.eval()
            images = sample['image']
            images = images.to(device)
            probs  = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)

            # 예측 결과를 
            # prediction_array에 입력
            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0],:]\
                         = preds.astype(int)
                         
    # 채널을 하나 추가하여 list에 append
    predictions_list.append(prediction_array[...,np.newaxis])

In [ ]:
# axis = 2를 기준으로 평균
predictions_array = np.concatenate(predictions_list, axis = 2)
predictions_mean = predictions_array.mean(axis = 2)

# 평균 값이 0.5보다 클 경우 1 작으면 0
predictions_mean = (predictions_mean > 0.5) * 1
predictions_mean

In [ ]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Data/sample_submission.csv")
sample_submission.iloc[:,1:] = predictions_mean
sample_submission.to_csv("baseline_prediction.csv", index = False)
sample_submission

In [ ]:
from google.colab import files

files.download("som_multilable_prediction_1".csv")